# The Kopi Latte Ratio Project: Data Collection

The objective of this notebook is to collect location & reviews data of coffee shops and kopitiams across Singapore by querying the Google Maps Places API.

## Get details of planning areas in Singapore using OneMap API

The Urban Redevelopment Authority (URA) has delineated a total of 55 planning areas in Singapore, which can be used in the Google Maps queries. Here, I am using the OneMap API to pull the names and geographic data of the planning areas to use in the analysis.

### Load Credentials & Get OneMap Access Token

In [1]:
import requests
from dotenv import load_dotenv
import os

# Load onemap credentials from .env file in root folder
load_dotenv()
onemap_email = os.getenv("ONEMAP_ACCOUNT_EMAIL")
onemap_password = os.getenv("ONEMAP_ACCOUNT_PASSWORD")

# Get auth token with POST request      
url = "https://www.onemap.gov.sg/api/auth/post/getToken"
      
payload = {
        "email":onemap_email,
        "password": onemap_password
      }
      
response = requests.request("POST", url, json=payload)
      
access_token = response.json().get('access_token')

### Request List of Planning Areas and GeoJSON Polygons

In [2]:
import pandas as pd
import json


url = "https://www.onemap.gov.sg/api/public/popapi/getAllPlanningarea?year=2019"
      
headers = {"Authorization": access_token}
      
response = requests.request("GET", url, headers=headers)

# The response is a list of planning area names and their respective geojson polygons. Parse the response and save as a pandas DataFrame
planning_areas = pd.DataFrame(response.json()['SearchResults'])
planning_areas.head()

,pln_area_n,geojson
0,BEDOK,"{""type"":""MultiPolygon"",""coordinates"":[[[[103.9..."
1,BUKIT TIMAH,"{""type"":""MultiPolygon"",""coordinates"":[[[[103.7..."
2,BUKIT BATOK,"{""type"":""MultiPolygon"",""coordinates"":[[[[103.7..."
3,BUKIT MERAH,"{""type"":""MultiPolygon"",""coordinates"":[[[[103.8..."
4,CENTRAL WATER CATCHMENT,"{""type"":""MultiPolygon"",""coordinates"":[[[[103.8..."


## Query Google Maps Places API for Location & Review Data

With the list of planning areas, the next step is to prepare the queries to request from the Google Maps API. My approach is to concatenate the location type (eg 'cafe') with each planning area (eg 'BUKIT TIMAH'). This will give us a comprehensive list of queries (eg 'cafe in BUKIT TIMAH') to extract the location data

### Prepare location queries for Google Maps Places API

In [3]:
def prepare_queries(queries, locations):
    """
    Prepare a list of queries for Place Search 

    Parameters:
        queries (list): A list of queries to search for. Eg ["coffee", "coffee shops", "kopi"]
        locations (list): A list of locations to search in for each query in query_list. Eg ["bishan", "orchard", "ang mo kio"]

    Returns: A list of queries after concatenating queries and locations. Eg 'coffee in bishan'
    """

    return [i + " in " + j for i in query_list for j in location_list]

In [4]:
query_list = ['coffee', 'coffee shop', 'kopi', 'kopitiam', 'cafe', 'latte']
location_list = planning_areas['pln_area_n']

queries = prepare_queries(query_list, location_list)
queries[0:2] + queries[-2:]

['coffee in BEDOK',
 'coffee in BUKIT TIMAH',
 'latte in SUNGEI KADUT',
 'latte in YISHUN']

### Query location data from Google Maps Places API

Each place search query returns a maximum of 60 results per query with the responses paginated with 20 results per response. Hence, I defined a function to return all 60 results in a query.

In [8]:
import time 

def get_all_query_results(query, gmaps_client):
    """
    Get maximum number of results (60) for each Google Place API Text Search query
    """
    results = []

    # Search for places using query and save value of next_page_token
    print("Requesting results for", query)
    response = gmaps_client.places(query=query)
    results.extend(response.get('results'))

    next_page_token = response.get("next_page_token")

    while next_page_token is not None:
        time.sleep(2)
        response = gmaps_client.places(page_token=next_page_token)
        results.extend(response.get("results"))
        next_page_token = response.get("next_page_token")
    else:
        print("next_page_token not found. Returning list of jsons for", query)

    return results


In [9]:
for query in queries[0:5]:
    print(query)

coffee in BEDOK
coffee in BUKIT TIMAH
coffee in BUKIT BATOK
coffee in BUKIT MERAH
coffee in CENTRAL WATER CATCHMENT


In [10]:
import googlemaps

load_dotenv()
places_api_key = os.getenv("PLACES_API_KEY")

# Initiate Google Maps API client
gmaps = googlemaps.Client(key=places_api_key)

# loop through queries and add to results list
results = []
for query in queries[0:2]:
    results.extend(get_all_query_results(query, gmaps))

Requesting results for coffee in BEDOK
next_page_token not found. Returning list of jsons for coffee in BEDOK
Requesting results for coffee in BUKIT TIMAH
next_page_token not found. Returning list of jsons for coffee in BUKIT TIMAH


In [20]:
# show available fields for each result
print(results[0].keys())

dict_keys(['business_status', 'formatted_address', 'geometry', 'icon', 'icon_background_color', 'icon_mask_base_uri', 'name', 'opening_hours', 'photos', 'place_id', 'plus_code', 'price_level', 'rating', 'reference', 'types', 'user_ratings_total'])


In [66]:
# There may be duplicate places across queries. Save response as pandas df and remove duplicates
places_df = pd.json_normalize(results).drop_duplicates(['place_id'])[[ 'name', 'place_id', 'formatted_address', 'user_ratings_total']]
places_df.head()

,name,place_id,formatted_address,user_ratings_total
0,Percolate,ChIJV6HD-Eo92jERjhfY7NEDrOM,"136 Bedok North Ave 3, #01-152, Singapore 460136",1007
1,Generation Coffee Roasters (Bedok),ChIJhbwWY-I92jERxtB-gF22sL0,"216 Bedok North Street 1, #01-32, Singapore 46...",200
2,Refuel Cafe,ChIJcf_SpPk82jERM28p3SYNBnI,"744 Bedok Reservoir Rd, #01-3029 Reservoir Vil...",1146
3,Dutch Colony Coffee Co.,ChIJF2GIiKYi2jERAGbOklC40B8,"113 Frankel Ave, Singapore 458230",1251
4,Marie's Lapis Cafe,ChIJ3Vc6OY092jERhObI1bZ_4Sk,"537 Bedok North Street 3, #01-575, Singapore 4...",379


# Get reviews for places

In [67]:
# Get a deduplicated list of place IDs for all places with at least one review in df
review_place_ids = places_df.loc[places_df["user_ratings_total"] > 0, "place_id"].drop_duplicates().tolist()


# Request for reviews in review_place_ids
reviews = []
for place_id in review_place_ids:
    response = gmaps.place(place_id)
    reviews.append(response)

# show available fields for each response
reviews[0]['result'].keys()

dict_keys(['address_components', 'adr_address', 'business_status', 'curbside_pickup', 'current_opening_hours', 'delivery', 'dine_in', 'editorial_summary', 'formatted_address', 'formatted_phone_number', 'geometry', 'icon', 'icon_background_color', 'icon_mask_base_uri', 'international_phone_number', 'name', 'opening_hours', 'photos', 'place_id', 'plus_code', 'price_level', 'rating', 'reference', 'reservable', 'reviews', 'serves_beer', 'serves_breakfast', 'serves_brunch', 'serves_dinner', 'serves_lunch', 'takeout', 'types', 'url', 'user_ratings_total', 'utc_offset', 'vicinity', 'website', 'wheelchair_accessible_entrance'])

In [68]:
# normalise each response into a dataframe with each review text as one row and assign 'place_id' column
reviews_normalized = [pd.json_normalize(place['result'], 'reviews').assign(place_id=place['result']['place_id']) for place in reviews]
reviews_df = pd.concat(reviews_normalized)[['place_id', 'text']]
reviews_df.head()

,place_id,text
0,ChIJV6HD-Eo92jERjhfY7NEDrOM,Nice chill cafe in the heartlands. I like the ...
1,ChIJV6HD-Eo92jERjhfY7NEDrOM,THE BANANA LOAF is a showstopper. I was initia...
2,ChIJV6HD-Eo92jERjhfY7NEDrOM,It’s great to find a cafe that serves good cof...
3,ChIJV6HD-Eo92jERjhfY7NEDrOM,Nice little independent cafe in Bedok. Quite a...
4,ChIJV6HD-Eo92jERjhfY7NEDrOM,Met my friends for coffee at Percolate. Menu l...
